In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train_set = datasets.MNIST(root = "./data", train=True, download=True, transform=None)
test_set = datasets.MNIST(root = "./data", train=False, download = True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [31]:
train_arrays = []
train_labels = []
test_arrays = []
test_labels = []

def pil_to_np(stock_set):
  target_threshold = []
  target_set = []
  target_labels = []
  for i in range(0, len(stock_set)):
    img = np.array(stock_set[i][0], dtype = np.uint8)
    thresh , im_bw = cv2.threshold(img,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    target_threshold.append(thresh)
    target_set.append(np.array(im_bw, dtype = np.uint8))
    target_labels.append(np.array(stock_set[i][1]))

  return target_threshold, target_set, target_labels

train_thresh, train_arrays, train_labels = pil_to_np(train_set)
test_thresh, test_arrays, test_labels = pil_to_np(test_set)

In [4]:
train_arrays[0].shape

(28, 28)

In [ ]:
### Creating contour maps from grayscale image

def gray_to_contours(dataset):
  contours = []
  for i in range(0, len(dataset)):
    contour, hierarchy = cv2.findContours(image=dataset[i], mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contour_transform = np.zeros((dataset[i].shape))
    contour_transform = cv2.drawContours(image = contour_transform, contours = contour , contourIdx=-1, color=(255))
    contours.append(contour_transform)
  
  return contours

train_contours = gray_to_contours(train_arrays)
test_contours = gray_to_contours(test_arrays)

In [42]:
class Data_Maps():
  def __init__(self, dataset):
    self.dataset = dataset

  def left_maps(self, rstep, cstep):
    sz = len(self.dataset)
    left_maps = []
    for i in range(sz):
      lmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(0,self.dataset[i].shape[1],cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        lmap.append(count)
      left_maps.append(lmap)
    return left_maps

  def right_maps(self, rstep, cstep):
    sz = len(self.dataset)
    right_maps = []
    for i in range(sz):
      rmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(self.dataset[i].shape[1]-1,-1,-1 * cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        rmap.append(count)
      right_maps.append(rmap)
    return right_maps

  def top_maps(self, rstep, cstep):
    sz = len(self.dataset)
    top_maps = []
    for i in range(sz):
      tmap = []
      for col in range(0,self.dataset[i].shape[1],cstep):
        count = 0
        for row in range(0,self.dataset[i].shape[0], rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        tmap.append(count)
      top_maps.append(tmap)
    return top_maps
  
  def bottom_maps(self, rstep, cstep):
    sz = len(self.dataset)
    b_maps = []
    for i in range(sz):
      bmap = []
      for col in range(0,self.dataset[i].shape[0],cstep):
        count = 0
        for row in range(self.dataset[i].shape[1]-1,-1,-1 * rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        bmap.append(count)
      b_maps.append(bmap)
    return b_maps

In [43]:
### Testing
m = Data_Maps(train_arrays)

l = m.left_maps(1,1)
r = m.right_maps(1,1)
t = m.top_maps(1,1)
b = m.bottom_maps(1,1)


In [46]:
l_22 = m.left_maps(2,2)
r_22 = m.right_maps(2,2)
t_22 = m.top_maps(2,2)
b_22 = m.bottom_maps(2,2)

In [50]:
l_23 = m.left_maps(2,3)
r_23 = m.right_maps(2,3)
t_23 = m.top_maps(3,2)
b_32 = m.top_maps(3,2)

[10, 10, 8, 8, 8, 7, 2, 2, 2, 2, 2, 2, 10, 10]